In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
#!pip install langgraph-checkpoint-sqlite

In [9]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict,Annotated,List
import operator
from langchain_core.messages import AnyMessage,SystemMessage,HumanMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import ToolMessage
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [11]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [12]:
model = ChatOpenAI(model="gpt-4o",temperature=0)

In [19]:

PLAN_PROMPT="""You are an expert writer tasked with creating a comprehensive essay outline.  
Write such an outline for the user-provided topic, including key points and instructions for each section to guide the writing process.
"""

In [20]:
WRITER_PROMPT="""You are an essay assistant tasked with writing excellince.\
Generate the best essay possible for the user's request and the initial essay.\
If the user provides critique, respond with a revised version of your provided essay.\
Utilize all the information below as needed:

------

{content}"""

In [21]:
REFLECTION_PROMPT="""You are a teacher grading an essay submission.\
Generate critique and recommendations for the user's submission.\
Provide detailed recommendations, including requests for length, depth, detials etc.\
"""

In [22]:
RESEARCH_PLAN_PROMPT="""You are a researcher tasked with providing relevant information to support the writing of the following essay. 
Generate a list of up to three search queries that will help in gathering necessary data and insights. 
Ensure that the queries are specific and targeted to yield useful results for the user's topic.
"""

In [25]:
RESEARCH_CRITIQUE_PROMPT="""You are a researcher charged with providing constructive feedback on an essay submission. 
Generate a list of search queries that will gather relevant information to support any requested revisions. 
Your queries should be specific, targeted, and designed to yield useful results that enhance the quality of the user's essay. 
Additionally, provide recommendations for areas of improvement based on the content of the essay.
"""

In [1]:
from langchain_core.pydantic_v1 import BaseModel
class Queries(BaseModel):
    queries: List[str]

c:\Users\user\miniforge3\Lib\site-packages\IPython\core\interactiveshell.py:3546: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


NameError: name 'List' is not defined

In [28]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]
    response=model.invoke(messages)
    return {"plan": response.content}